In all versions 2.x the narrative gets cleaned as before and products are cleaned twice. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time the same happens with the six most occuring ones. This run uses 10 different principle components.

__Initialing the data__

In [8]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_both.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 3


__Initializing the bag of words__

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [10]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=10)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(208872, 10)


In [11]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.18578621 -3.61074476  0.13379118 ... -0.47769417 -0.72143028
   0.83083891]
 [ 3.99741845 -1.1975272   0.72272862 ... -0.18135558 -0.12623301
  -0.11902317]
 [ 2.08671497  0.90268685  2.29909557 ... -0.03070652  0.66966474
   0.79917863]
 ...
 [ 8.09025727 -8.10500172 -1.75889552 ... -1.68269286 -2.31766757
   1.70611601]
 [ 5.24479353 -5.0118109  -0.97536482 ... -1.20185751 -0.8614178
  -0.65661415]
 [ 3.81963549 -2.15467603  0.5804621  ... -0.37329915 -1.53251186
   0.49740042]]
                0          1         2         3         4         5  \
0        4.185786  -3.610745  0.133791  0.789644  0.120347 -0.061496   
1        3.997418  -1.197527  0.722729 -0.687870  0.304083  0.013298   
2        2.086715   0.902687  2.299096 -0.951874 -0.480134  0.328047   
3       24.306424 -15.933717  3.726543  8.877398 -9.840651  3.538600   
4        3.268770   0.855807  2.254020 -1.288967 -0.392812 -0.068501   
...           ...        ...       ...       ...       ...       ...   
20886

__One hot encoding of extra columns__

In [12]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(208872, 10)
(208872, 161)
(208872, 171)


__Split into train and test__

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(156654, 171)
(52218, 171)
(156654,)
(52218,)


__Application of linear SVM__

In [14]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3721
                                                 Checking or savings account       0.99      1.00      1.00      4791
                                                               Consumer Loan       0.94      0.78      0.85      2882
                                                                 Credit card       1.00      1.00      1.00      4675
                                                 Credit card or prepaid card       0.95      0.96      0.96      4579
                                                            Credit reporting       1.00      1.00      1.00      4721
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.86      0.92      6102
                                                       